In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
Source_Data = [
    ("1","Hemanth","HYd","506002")
        ,("2","Raj","BLR","560001")
        ,("3","Kumar","MUM","400001")
        ,("4","Ravi","BLR","560001")
        ]

DSchema  = StructType(
    [StructField("Id",StringType(),True),
     StructField("Name",StringType(),True),
     StructField("City",StringType(),True),
     StructField("Pin",StringType(),True)]
    )
                       

In [0]:
df_Source = spark.createDataFrame(Source_Data,DSchema)
df_Source.

df_Source.display()

In [0]:
df_target = df_Source.withColumn("Valid_date",current_timestamp()).withColumn("Valid_to",lit(None).cast("timestamp")).withColumn("Vaild_Flag",lit("Y"))
df_target.display()

In [0]:
updated_Data = [
    ("1","Hemanth","wgl","506001")
        ,("2","Raj","BLR","560001")
        ,("3","Kumar","MUM","400001")
        ,("4","Ravi","BLR","560001")
        ,("5","Raju","Chn","560008")
        ]

df_update = spark.createDataFrame(updated_Data,DSchema)
df_update.display()

In [0]:


df_target.write.format("delta").saveAsTable("default.CustomerData")

# Drop Table if exists default.CustomerData


In [0]:
%sql
select * from default.customerdata

In [0]:
from delta.tables import DeltaTable


In [0]:
target_table = DeltaTable.forName(spark, "default.CustomerData")

In [0]:
# df_join = df_update.alias("Newdata").join(df_target.alias("OldData"),df_update.Id == df_target.Id,"inner")\
# .filter(df_target.Vaild_Flag == "Y")\
# .filter(df_update.Pin != df_target.Pin)\
# .filter(df_target.City != df_update.City)


# df_join.display()



df_join  = df_update.join(target_table.toDF() ,df_update.Id == target_table.toDF().Id,"inner")\
.filter(target_table.toDF().Vaild_Flag == "Y")\
.filter(df_update.Pin != target_table.toDF().Pin)\
.filter(target_table.toDF().City != df_update.City)
df_join.display()

In [0]:
df_customerId = df_join.select(df_update.Id)
df_customerId.display()


In [0]:
target_table.alias("t").merge( 
                              source=df_customerId.alias("CusID"),
                            condition="t.Id = CusID.Id and t.Vaild_Flag = 'Y'"
    ).whenMatchedUpdate(set = 
                        {
                            "Vaild_Flag" : lit("N"),
                            "Valid_to" : current_timestamp()
                         }
                        ).execute()

In [0]:
%sql
select *  from customerdata

In [0]:
source.display()

In [0]:

new_records = df_target.join(
    df_customerId, 
    df_target.Id == df_customerId.Id, 
    "left_semi"
).withColumn("Valid_date", current_timestamp()
).withColumn("Valid_to", lit(None).cast("timestamp")
).withColumn("Valid_Flag", lit("Y"))

new_records.display()
